In [211]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pickle

In [212]:
url = 'https://raw.githubusercontent.com/dandia14/project-capstone-team-csd-134/ml/dataset/diabetes.csv'
diabetes = pd.read_csv(url)
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [213]:
cols = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
for col in cols:
    diabetes[col].replace(0,np.NaN,inplace=True)

In [214]:
# Mengganti nilai NaN dengan nilai median sesuai target Outcome
for col in diabetes.columns:
    diabetes.loc[(diabetes["Outcome"]==0) & (diabetes[col].isnull()),col] = diabetes[diabetes["Outcome"]==0][col].median()
    diabetes.loc[(diabetes["Outcome"]==1) & (diabetes[col].isnull()),col] = diabetes[diabetes["Outcome"]==1][col].median()

In [215]:
Q1 = diabetes.quantile(0.25)
Q3 = diabetes.quantile(0.75)
IQR=Q3-Q1
diabetes=diabetes[~((diabetes<(Q1-1.5*IQR))|(diabetes>(Q3+1.5*IQR))).any(axis=1)]
 
# Cek ukuran dataset setelah kita drop outliers
diabetes.shape

(590, 9)

In [216]:
X = diabetes.values
X = np.delete(X, 8, axis=1)
y = diabetes['Outcome']

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [218]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
# Number of faetures to consider at every split
max_features = ['auto', 'sqrt']
# Maximun number of levels in tree
max_depth = [2, 4]
# Minimum number of samples required to split a node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [219]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}
print(param_grid)

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [220]:
rf_Model = RandomForestClassifier(random_state=0)

In [221]:
rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid, cv = 3, verbose = 2, n_jobs = -1)

In [222]:
rf_Grid.fit(X_train, y_train)

Fitting 3 folds for each of 160 candidates, totalling 480 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]},
             verbose=2)

In [223]:
rf_Grid.best_params_

{'bootstrap': True,
 'max_depth': 4,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'n_estimators': 40}

In [224]:
print(f'Train Accuracy - : {rf_Grid.score(X_train, y_train):.3f}')
print(f'Test Accuracy - : {rf_Grid.score(X_test, y_test):.3f}')

Train Accuracy - : 0.934
Test Accuracy - : 0.873


In [225]:
pickle.dump(rf_Grid, open('model_rfc.pkl', 'wb'))

In [226]:
location = './model_rfc.pkl'
file = open(location, "rb")
model = pickle.load(file)

In [227]:
data1 = [10,	168,	74,	32,	169.5,	38,	0.537,	34]
data2 = [4,	110,	92,	27,	102.5,	37.6,	0.191,	30]
data1 = np.array(data1)
data2 = np.array(data2)

data1 = data1.reshape(1,-1)
data2 = data2.reshape(1,-1)
print(data1)
print(data2)

[[ 10.    168.     74.     32.    169.5    38.      0.537  34.   ]]
[[  4.    110.     92.     27.    102.5    37.6     0.191  30.   ]]


In [228]:
prediction1 = model.predict(data1)
prediction2 = model.predict(data2)

print(prediction1)
print(prediction2)

[1]
[0]
